In [1]:
import os
import zipfile
import shapefile
import pandas as pd
from simpledbf import Dbf5
import dbf
import csv

In [2]:
base_path = 'D:/URBAG/Catastro/OneDrive_2020-07-24/Rustico actual/'

In [25]:
folders = next(os.walk(base_path))[1]
folders

['08_100_RA_2020-04-25_SHF',
 '08_122_RA_2020-04-24_SHF',
 '08_124_RA_2020-04-24_SHF',
 '08_125_RA_2020-04-25_SHF',
 '08_156_RA_2020-04-24_SHF',
 '08_168_RA_2020-04-24_SHF',
 '08_179_RA_2020-04-24_SHF',
 '08_193_RA_2020-04-24_SHF',
 '08_195_RA_2020-04-24_SHF',
 '08_199_RA_2020-04-24_SHF',
 '08_203_RA_2020-04-24_SHF',
 '08_204_RA_2020-04-24_SHF',
 '08_210_RA_2020-04-24_SHF',
 '08_216_RA_2020-04-24_SHF',
 '08_219_RA_2020-04-24_SHF',
 '08_244_RA_2020-04-24_SHF',
 '08_245_RA_2020-04-24_SHF',
 '08_263_RA_2020-04-24_SHF',
 '08_282_RA_2020-04-25_SHF',
 '08_289_RA_2020-04-25_SHF',
 '08_302_RA_2020-04-25_SHF',
 '08_313_RA_2020-04-25_SHF',
 '08_76_RA_2020-04-24_SHF',
 '08_88_RA_2020-04-25_SHF',
 'Badalona08_15_RA_2020-04-24_SHF',
 'Badia08_312_RA_2020-04-25_SHF',
 'Barbera08_252_RA_2020-04-24_SHF',
 'Barcelona08_900_RA_2020-04-24_SHF',
 'Begues08_20_RA_2020-04-25_SHF',
 'Castellbisbal08_53_RA_2020-04-25_SHF',
 'Castelldefelds08_55_RA_2020-04-25_SHF',
 'Cerdanyola08_266_RA_2020-04-25_SHF',
 'Cerv

In [4]:
def unzipFile(origin, destination):
    with zipfile.ZipFile(origin,"r") as zip_ref:
        zip_ref.extractall(destination)

In [5]:
def unzipCatastro(base_folder):
    folders = next(os.walk(base_path))[1]
    for folder in folders:
        unzipFile(base_folder+folder+'/SUBPARCE.zip',base_folder+folder+'/SUBPARCE')
        unzipFile(base_folder+folder+'/RUSUBPARCELA.zip',base_folder+folder+'/RUSUBPARCELA')
        unzipFile(base_folder+folder+'/RUCULTIVO.zip',base_folder+folder+'/RUCULTIVO')

In [6]:
unzipCatastro(base_path)

In [39]:
i = 1
for folder in folders:
    subparce_file = base_path+folder+'/SUBPARCE/SUBPARCE.DBF'
    subparce_dbf = Dbf5(subparce_file)
    subparce_DF = subparce_dbf.to_dataframe()

    rusubparcela_file = base_path+folder+'/RUSUBPARCELA/RUSUBPARCELA.DBF'
    rusubparcela_dbf = Dbf5(rusubparcela_file)
    rusubparcela_DF = rusubparcela_dbf.to_dataframe()

    rucultivo_file = base_path+folder+'/RUCULTIVO/RUCULTIVO.DBF'
    rucultivo_dbf = Dbf5(rucultivo_file, codec='cp1250')
    rucultivo_DF = rucultivo_dbf.to_dataframe()

    subparce_DF['REFCAT_SUBPARCE'] = subparce_DF['REFCAT'] + subparce_DF['SUBPARCE']
    rusubparcela_DF['REFCAT_SUBPARCE'] = rusubparcela_DF['REFCAT'] + rusubparcela_DF['SUBPARCELA']

    subparce_rusubparcela_DF = pd.merge(subparce_DF, rusubparcela_DF, how='left', on='REFCAT_SUBPARCE')
    subparce_rusubparcela_rucultivo_DF = pd.merge(subparce_rusubparcela_DF, rucultivo_DF, how='left', on='CC')
    clean_subparce_rusubparcela_rucultivo_DF =subparce_rusubparcela_rucultivo_DF.rename(columns={"MUNICIPIO_x":"MUNICIPIO", "MASA_x":"MASA", "HOJA_x":"HOJA", "PARCELA_x":"PARCELA", "PCAT1_x":"PCAT1", "PCAT2_x":"PCAT2", "REFCAT_x":"REFCAT"})
    final_DF = clean_subparce_rusubparcela_rucultivo_DF.drop(['SUBPARCE','DELEGACIO_x', 'DELEGACIO_y', 'MUNICIPIO_y', 'REFCAT_y', 'PCAT1_y', 'PCAT2_y', 'HOJA_y', 'MASA_y', 'PARCELA_y'], axis=1)
    final_DF.to_csv(base_path+'../csv'+'/'+folder+'SUBPARCE.csv', quotechar='"', sep=',', index=False )

In [50]:
base_csv_path = 'D:/URBAG/Catastro/OneDrive_2020-07-24/csv/'

In [51]:
csv_files = os.listdir(base_csv_path)

In [52]:
df_array = []
for file in csv_files:
    df_array.append(pd.read_csv(base_csv_path+file))
Full_DF = pd.concat(df_array) 

In [54]:
Full_DF.to_csv(base_path+'../csv'+'/FULL_SUBPARCE.csv', quotechar='"', sep=',', index=False )